In [ ]:
# SG4 - FTN
# Topic: facial expression data preprocess & abstraction

In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import itertools 

In [ ]:
# Path
dir_present = '...' #present directory
destination_folder = dir_present+"/data/result" #result 
path_clean = dir_present+"/data/clean/SG4" #clean folder
clean_file = os.walk(path_clean) 

In [ ]:
# Create blank csv
df1 = pd.DataFrame(columns = range(7))
df1.columns = ["user_id", "game_id", "round_id", "device_id", "time_slot", "sample_size", "block_number"]
save_info_path = destination_folder + '/SG4_info_FTN_1s.csv'
df1.to_csv(save_info_path, index=False, header=True) #default: header=False

df2 = pd.DataFrame(columns = range(325)) # 5  + 5 * 64 = 325 (* 64 points in face; w0-w63)
df2_str_list = []
for j in ["max", "min", "mean", "std", "med"]:
  for i in range(64):
    df2_str_list.append("w" + str(i) + "_" + j)
df2_id_list = ["user_id", "game_id", "round_id", "device_id","block_id"]
df2_str_list = df2_id_list + df2_str_list 
df2.columns = df2_str_list
save_feat_path = destination_folder + '/SG4_fuse_ts_feature_FTN_1s.csv'  
df2.to_csv(save_feat_path, index=False, header=True)

In [ ]:
# Divide block and abstract the information

for path, dir_list, file_list in os.walk(dir_present+"/data/clean/SG4"): #cleaned dir
  for file_name in file_list:

    filepath = os.path.join(path, file_name)
    raw = pd.read_csv(filepath)
    raw_data = raw.loc[:,"Weights[0]":"Weights[63]"] # 64 features

    # get FBN_g 
    # FBN_g : how many blocks this ts need to be divided into
    str2 = file_name.split(".")
    str3 = str2[0].split("_")
    id_list = [int(element) for element in str3]
    g_id = int(id_list[1]) 

    # Build ts blocks
    TS = 1 # TS = 1 means around one second per block
    LEN = raw.shape[0]
    ts_delta = (int(raw.at[LEN-1, "Timestamp"]) - int(raw.at[0, "Timestamp"])) / pow(10,9)  
    sample_rate = int(LEN/ts_delta) #floor
    granu = int(TS * sample_rate)
    df_wr = pd.DataFrame() 

    # Traverse Blocks in a ts
    for i in range(1, int(ts_delta/TS) + 1):
      
      #set block
      raw_block = raw_data[(i-1)*granu : (i*granu-1)]
      
      #compute statistics
      fea_block = [] # 5 statistics
      fea_block += list(raw_block.max())
      fea_block += list(raw_block.min())
      fea_block += list(raw_block.mean())
      fea_block += list(raw_block.std())
      fea_block += list(raw_block.median())
      df_block_fea = pd.DataFrame(fea_block)
      df_block_fea = df_block_fea.T
      
      #combine blocks feature
      str2 = file_name.split(".")
      str3 = str2[0].split("_")
      id_list = [int(element) for element in str3]
      id_list.append(i)
      
      #print(id_list)
      id_df = pd.DataFrame(id_list)
      id_df = id_df.T
      id_df.columns = ["user_id", "game_id", "round_id", "device_id", "block_id"]
      df_block_row = pd.concat([id_df, df_block_fea], axis = 1, join = 'outer')
      df_wr = df_wr.append(df_block_row)
      
    
    # Save One file info
    str2 = file_name.split(".")
    str3 = str2[0].split("_")
    id_list = [int(element) for element in str3]

    id_list.append(ts_delta) # sec info: how many seconds
    id_list.append(LEN) # how many samples
    id_list.append(i+1) # how many blocks
    id_df = pd.DataFrame(id_list)
    id_df = id_df.T
    id_df.columns = ["user_id", "game_id", "round_id", "device_id", "time_slot", "sample_size", "block_number"]
    id_df.to_csv(save_info_path, mode='a', index=False, header=False)
    #time_slot: how long for user playing the game: Unit-second
    #sample_size:how many ts data
    #block_number:timeseries is divided into how many blocks
    
    # Save file featues
    df_wr.to_csv(save_feat_path, mode='a', index=False, header=False)